## Present proof

Presenting a proof involves establishing a connection between the issuer and the holder, which is done by following the process outlined in `Example 01 - Connections`. Once the connection is established, the verifier will prepare and send a proof request, which creates a presentation record on both the verifier's and holder's agents. The holder will then retrieve the list of presentations, find the one they wish to accept, and notify the verifier of their acceptance. The accept proof message contains an id of a credential stored in the Holders Agent. Finally, the Verifier will receive the proof from the holder, completing the process.

Note: the terminology "proof request" and "presentation request" may be used interchangeably 

### ⚠️ Important Note 
Please run *Example 03 - Issue Credential* before continuing with this example. 

In [ ]:
#🚨 Run this code cell to import requirements in the Kernel

import os
import time
import datetime
import base64
from pprint import pprint
from dotenv import load_dotenv

from prism_agent_open_api_specification_client import Client
from prism_agent_open_api_specification_client.types import Response, Unset
from prism_agent_open_api_specification_client.models import ConnectionCollection,Connection,ConnectionInvitation,CreateConnectionRequest,AcceptConnectionInvitationRequest
from prism_agent_open_api_specification_client.api.connections_management import get_connections,get_connection,create_connection,accept_connection_invitation
from prism_agent_open_api_specification_client.models import IssueCredentialRecord, CreateIssueCredentialRecordRequest, IssueCredentialRecordCollection, IssueCredentialRecordAllOfProtocolState
from prism_agent_open_api_specification_client.api.issue_credentials_protocol import get_credential_record, get_credential_records, create_credential_offer,accept_credential_offer,issue_credential
from prism_agent_open_api_specification_client.models import PresentationStatus, ErrorResponse, Proof, ProofRequestAux, PublicKeyJwk, RequestPresentationInput, RequestPresentationOutput, RequestPresentationAction, RequestPresentationActionAction
from prism_agent_open_api_specification_client.api.present_proof import get_presentation, get_all_presentation, request_presentation, update_presentation

### Ultilitary functions

In [ ]:
def get_invitation_str(connection):
    parts = connection.invitation.invitation_url.split("=")
    return parts[1]

def find_proof_request_by_state(client, state):
    proof_requests: Response[PresentationStatus] = get_all_presentation.sync(client=client)
    #proof_requests.reverse()
    for proof_request in proof_requests:
        if(proof_request.status == state):
            return proof_request
    return None 

def find_credential(client):
    credential_records: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=client)
    for offer in credential_records.contents:
        if not (type(offer.jwt_credential) is Unset):
            return offer
    return None 

def print_proof_request(proof_request):
    if hasattr(proof_request, "presentation_id"):
        print(f"presentation_id: {proof_request.presentation_id}")
    if hasattr(proof_request, "status"):
        print(f"status:          {proof_request.status}")
    if hasattr(proof_request, "connection_id"):
        print(f"connection_id:   {proof_request.connection_id}")

def print_connection(connection):
    print(f"connection_id: {connection.connection_id}")
    print(f"state:         {connection.state}")
    print(f"label:         {connection.label}")
    print(f"my_did:        {connection.my_did}")
    print(f"their_did:     {connection.their_did}")
    print(f"created_at:    {connection.created_at}")

### Client instances

We will create two separate clients, one for the Verifier and one for the Holder, in order to establish a connection between the two.

⚠️ Remember to update the file variables.env with the URLs and API keys provided to you.


In [ ]:
load_dotenv("../BetaProgram/variables.env")
verifierApiKey = os.getenv('VERIFIER_APIKEY')
verifierUrl = os.getenv('VERIFIER_URL')

holderApiKey = os.getenv('HOLDER_APIKEY')
holderUrl = os.getenv('HOLDER_URL')

verifier_client = Client(base_url=verifierUrl, headers={"apiKey": verifierApiKey})
holder_client = Client(base_url=holderUrl, headers={"apiKey": holderApiKey})

print(f"Verifier URL:{verifierUrl}")
print(f"Holder URL:{holderUrl}")

### Create connection

ℹ️ For details on this see "Example 01 - Connections"

In [ ]:
print("Please wait...")

conn_request = CreateConnectionRequest()
conn_request.label = f'Present proof {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
verifier_connection: Response[Connection] =  create_connection.sync(client=verifier_client,json_body=conn_request)

invitation = get_invitation_str(verifier_connection)

accept_conn_request = AcceptConnectionInvitationRequest(invitation)
holder_connection: Response[ConnectionInvitation] =  accept_connection_invitation.sync(client=holder_client,json_body=accept_conn_request)

verifier_connection: Response[Connection] = get_connection.sync(client=verifier_client,connection_id=verifier_connection.connection_id)
holder_connection: Response[Connection] = get_connection.sync(client=holder_client,connection_id=holder_connection.connection_id)

while (verifier_connection.state != 'ConnectionResponseSent' or 
       not(holder_connection.state == 'ConnectionResponseReceived' or holder_connection.state == 'ConnectionRequestSent')):
    verifier_connection: Response[Connection] = get_connection.sync(client=verifier_client,connection_id=verifier_connection.connection_id)
    holder_connection: Response[Connection] = get_connection.sync(client=holder_client,connection_id=holder_connection.connection_id)
    print("Verifier State: {} / Holder State: {} \n".format(verifier_connection.state,holder_connection.state))
    time.sleep(1)
    
print("Connection established between verifier and Holder!")
print("\nVerifier connection:\n")
print_connection(verifier_connection)
print("\nHolder connection:\n")
print_connection(holder_connection)

### Verifier - Create proof request
The Verifier prepares the proof request, it uses the `connection_id` of the connection with the Holder to define where to send the request. The `proofs` describe the credential requested 

In [ ]:
data = {
    "description":"Request presentation of credential",
    "connectionId": verifier_connection.connection_id,
    "options":{
        "challenge": "11c91493-01b3-4c4d-ac36-b336bab5bddf",
        "domain": "https://example-verifier.com"
    },
    "proofs":[
        {
            "schemaId": "https://schema.org/Person",
            "trustIssuers": [
                "did:web:atalaprism.io/users/testUser"
            ]
        }
    ]
}

proof_request = RequestPresentationInput.from_dict(data)

### Verifier - Send proof request

The Verifier sends the proof request. This action creates the presentation record in the Verifier side and sends the request to the Holder using the connection

In [ ]:
verifier_proof_request: Response[RequestPresentationInput] = request_presentation.sync(client=verifier_client, json_body=proof_request)
print("\nVerifier proof request:\n")
print_proof_request(verifier_proof_request)

### Holder - Wait for proof request

The Holder waits to receive the request

In [ ]:
print("Please wait...")

holder_proof_request = find_proof_request_by_state(holder_client, "RequestReceived")

while(holder_proof_request == None):
    holder_proof_request = find_proof_request_by_state(holder_client, "RequestReceived")
    time.sleep(1)

print("\nHolder proof request:\n")
print_proof_request(holder_proof_request)

### Holder - Accept proof request

The Holder accepts the proof request by updating the presentation record with the action `REQUEST_ACCEPT`. The update also provides the `proof_id` corresponding to the credential used to fulfill the proof request.

⚠️ The program will prompt for a credential `record_id`. Provide the one obtained at the last step of *Example 04 - Issue Credential*

**Note: `record_id` and `proof_id` refer to the same value**

In [ ]:
def valid_credential(client, record_id):
    credential_record = get_credential_record.sync(client=client, record_id=record_id)
    if credential_record is None:
        return False
    elif type(credential_record.jwt_credential) is Unset:
        return False
    else:
        return True

while True:
    credential_record_id = input("\nprovide a credential record_id").strip()
    if credential_record_id == "":
        print(f"\n🚨 The provided credential is not valid. Please create a credential on this agent {holderUrl} to proceed")
    elif(valid_credential(holder_client, credential_record_id)):
        print(f"\n✅ Credential is valid: {credential_record_id}")
        break
    else:
        print(f"\n🚨 The provided credential is not valid. Please create a credential on this agent {holderUrl} to proceed")

action = RequestPresentationAction(action=RequestPresentationActionAction.REQUEST_ACCEPT, proof_id=[credential_record_id])

#Always returns none
update_presentation.sync(client=holder_client, json_body=action, id=holder_proof_request.presentation_id)

print("\nHolder proof request:\n")
print(holder_proof_request.presentation_id)

### Verifier - Wait for verification

The Verifier waits for the proof. Once received, it updates the status of the presentation and gets the verifiable presentation data

In [ ]:
print("Please wait...")

verifier_proof_request: Response[PresentationStatus] = get_presentation.sync(client=verifier_client, id=verifier_proof_request.presentation_id)

print(f"Verifier presentation: {verifier_proof_request.presentation_id}")
print(f"Holder presentation:   {holder_proof_request.presentation_id}\n")
while(verifier_proof_request.status != "PresentationVerified"):
    verifier_proof_request: Response[PresentationStatus] = get_presentation.sync(client=verifier_client, id=verifier_proof_request.presentation_id)
    holder_proof_request: Response[PresentationStatus] = get_presentation.sync(client=holder_client, id=holder_proof_request.presentation_id)
    print("Verifier State: {} / Holder State: {}".format(verifier_proof_request.status,holder_proof_request.status))
    time.sleep(1)
    
print_proof_request(verifier_proof_request)

### Verifier - Check the presentation

The website https://jwt.io/ can be used to decode the verifiable presentation.

In [ ]:
verifier_proof_request.data[0]
